In [ ]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the directory containing the PDF files
pdf_directory = './data'

논문을 벡터 db에 넣기

In [ ]:
# pdf를 사용해서 pdf(논문)을 모두 로드
pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))

# Load all PDF files using PyPDFLoader
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents = loader.load()
    documents.extend(pdf_documents)
    
# 텍스트는 RecursiveCharacterTextSplitter를 사용하여 분할
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = chunk_splitter.split_documents(documents)

# embeddings은 OpenAI의 임베딩을 사용
# vectordb는 chromadb사용함
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vectordb.as_retriever()

모델 선언

In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA

# Initialize the OpenAIEmbeddings and ChatOpenAI objects
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
chat_model = ChatOpenAI(api_key=OPENAI_API_KEY, model_name="gpt-4o", temperature=0)

# Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(llm=chat_model, chain_type="stuff", retriever=retriever, return_source_documents=True)

In [ ]:
# Create the QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(api_key=OPENAI_API_KEY, model_name="gpt-4o", temperature=0),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [1]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
# 환경 변수 불러오기
load_dotenv()

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the directory containing the PDF files
pdf_directory = './data'

# Load all PDF files using PyPDFLoader
pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))

for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents = loader.load()
    documents.extend(pdf_documents)

# 텍스트를 RecursiveCharacterTextSplitter를 사용하여 분할
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = chunk_splitter.split_documents(documents)

# Embedding 모델 초기화
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

# Chroma를 이용해 Vector DB 생성
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vectordb.as_retriever()

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model_name="gpt-4o", temperature=0)

# 시스템 프롬프트 설정
SYS_PROMPT = """
너는 사용자의 인적정보, KGLS 질문과 사용자답변, 사용자가 입력한 주관식 답변 그리고 논문들의 외로움에 관한 연구를 바탕으로 이와 관련된 외로움을 가지고 있는지 설정해주고 그 사용자에게 적절한 대화 상대가 되기 위한 프롬프트를 출력해줘.
"""
# 분석 요청 입력 프롬프트 설정
input_prompt = """
이 질문과 답변을 사용자의 외로움을 파악하기 위함이야. 논문에서 참고해서 사용자가 어떤 외로움을 느끼는건지, 어떤 대화상대가 필요할지 알아보고, 사용자가 경험한 외로움이 뭔지와 적절한 대화하기 위한 프롬프트를 출력해줘.
"""


# 예시 입력
example_input = {

    "인적정보": "홍길동, 35세, 남성",

    "KGLS질문": "최근에 외로움을 느꼈나요?",

    "주관식 질문": "외로움을 느낀 이유가 무엇인가요?"

}
input_prompt += example_input
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        ("human", "{input_prompt}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


ModuleNotFoundError: No module named 'langchain_community'